In [66]:
get_source_tool = {
    "type": "function",
    "function": {  # Nested under "function" key
        "name": "get_source_tool",
        "description": "Retrives source location:",
        "parameters": {
            "type": "object",
            "properties": {
                "source_location": {
                    "type": "string",
                    "description": "Provides source from where journey started. e.g. home, Mumbai, India"
                }
            },
            "required": ["source_location"],
            "additionalProperties": False
        }
    }
}

In [67]:
get_destination_tool = {
    "type": "function",
    "function": {  # Nested under "function" key
        "name": "get_destination_tool",
        "description": "Retrives destination location:",
        "parameters": {
            "type": "object",
            "properties": {
                "destination_location": {
                    "type": "string",
                    "description": "Provides destination where journey ended. e.g. home, Mumbai, India"
                }
            },
            "required": ["destination_location"],
            "additionalProperties": False
        }
    }
}

In [68]:
text="I am travelling from Ahmedabad to Pune"

In [69]:
from openai import OpenAI
import os

In [70]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
response3 = client.chat.completions.create(
    model='gpt-4o-mini',  # Valid model (gpt-4.1-nano doesn't exist)
    messages=[
        {"role": "system", "content": "You are a helpful assistant that extracts structured information from text."},
        {"role": "user", "content": text},
        {"role":}
    ],
    temperature=0,
    top_p=1,
    tools=[get_source_tool, get_destination_tool]  # Use 'tools' not 'functions' [web:182]
)

In [72]:
print(response)

ChatCompletion(id='chatcmpl-Cki8HEAUnAnioKwCvIxUcMI9F19uo', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_KbBYeinWrLB9Tj4yCGvd1wlZ', function=Function(arguments='{"source_location": "Ahmedabad"}', name='get_source_tool'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_EJHwLmIhCLKXTPSuPtZFE0Zk', function=Function(arguments='{"destination_location": "Pune"}', name='get_destination_tool'), type='function')]))], created=1765248769, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_aa07c96156', usage=CompletionUsage(completion_tokens=50, prompt_tokens=117, total_tokens=167, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0),

In [73]:
print(response.choices[0].message.tool_calls[0].function.arguments)

{"source_location": "Ahmedabad"}


In [74]:
print(response.choices[0].message.tool_calls[1].function.arguments)

{"destination_location": "Pune"}


In [75]:
import json

In [76]:
source=json.loads(response.choices[0].message.tool_calls[0].function.arguments).get("source_location")

In [35]:
source

'Ahmedabad'

In [78]:
destination=json.loads(response.choices[0].message.tool_calls[1].function.arguments).get("destination_location")

In [79]:
destination

'Pune'

In [80]:
import requests

address = source
url = f'https://nominatim.openstreetmap.org/search?q={address}&format=json&limit=1'
headers = {'User-Agent': 'YourApp/1.0'}  # Required
response = requests.get(url, headers=headers)
data = response.json()
lat, lon = data[0]['lat'], data[0]['lon']
source_boundingbox=data[0]['boundingbox']

In [81]:
source_boundingbox

['22.8615374', '23.1815374', '72.4200568', '72.7400568']

In [82]:

address = destination
url = f'https://nominatim.openstreetmap.org/search?q={address}&format=json&limit=1'
headers = {'User-Agent': 'YourApp/1.0'}  # Required
response = requests.get(url, headers=headers)
data = response.json()
lat, lon = data[0]['lat'], data[0]['lon']
destination_boundingbox=data[0]['boundingbox']

In [83]:
destination_boundingbox

['18.4294970', '18.6208699', '73.7498473', '74.0202140']

In [84]:
text =text + f" Source: {source}, source_boundingbox: {source_boundingbox}. Destination: {destination}, destination_boundingbox: {destination_boundingbox}."
response = client.chat.completions.create(
    model='gpt-4o-mini',  # Valid model (gpt-4.1-nano doesn't exist)
    messages=[
        {"role": "system", "content": "You are a helpful assistant that displays city and it's bounding box in table format."},
        {"role": "user", "content": text}
    ],
    temperature=0,
    top_p=1,
)

In [85]:
print(response)

ChatCompletion(id='chatcmpl-Cki9vks35O45UhGBC3IxedDUzW8sF', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Here is the information in table format for your travel from Ahmedabad to Pune:\n\n| City      | Bounding Box Coordinates                     |\n|-----------|----------------------------------------------|\n| Ahmedabad | ['22.8615374', '23.1815374', '72.4200568', '72.7400568'] |\n| Pune      | ['18.4294970', '18.6208699', '73.7498473', '74.0202140'] |\n\nSafe travels!", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1765248871, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_11f3029f6b', usage=CompletionUsage(completion_tokens=98, prompt_tokens=108, total_tokens=206, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), 

In [86]:
print(response.choices[0].message.content)

Here is the information in table format for your travel from Ahmedabad to Pune:

| City      | Bounding Box Coordinates                     |
|-----------|----------------------------------------------|
| Ahmedabad | ['22.8615374', '23.1815374', '72.4200568', '72.7400568'] |
| Pune      | ['18.4294970', '18.6208699', '73.7498473', '74.0202140'] |

Safe travels!
